In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!unzip '/content/gdrive/MyDrive/he1.zip' -d '/content/gdrive/MyDrive/Processed/Data'

Streaming output truncated to the last 5000 lines.
  inflating: /content/gdrive/MyDrive/Processed/Data/he1/981972resized4rotated_md_he.jpg  
  inflating: /content/gdrive/MyDrive/Processed/Data/he1/982013resized1rotated_md_he.jpg  
  inflating: /content/gdrive/MyDrive/Processed/Data/he1/982013resized2rotated_md_he.jpg  
  inflating: /content/gdrive/MyDrive/Processed/Data/he1/982013resized3rotated_md_he.jpg  
  inflating: /content/gdrive/MyDrive/Processed/Data/he1/982013resized4rotated_md_he.jpg  
  inflating: /content/gdrive/MyDrive/Processed/Data/he1/982018resized1rotated_md_he.jpg  
  inflating: /content/gdrive/MyDrive/Processed/Data/he1/982018resized2rotated_md_he.jpg  
  inflating: /content/gdrive/MyDrive/Processed/Data/he1/982018resized3rotated_md_he.jpg  
  inflating: /content/gdrive/MyDrive/Processed/Data/he1/982018resized4rotated_md_he.jpg  
  inflating: /content/gdrive/MyDrive/Processed/Data/he1/982076resized1rotated_md_he.jpg  
  inflating: /content/gdrive/MyDrive/Processed/Da

In [94]:
import keras
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda, Reshape
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Conv2D, MaxPooling2D, Activation
import keras.optimizers

In [95]:
def ConvLayers(layers, model, filters):
  for i in range(layers):   
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(filters, 3,3,activation='relu'))
  model.add(MaxPooling2D((2,2),strides=(1,1),padding="same"))

In [96]:
def FCLayers(model):
  model.add(Dense(4096, activation='relu'))
  model.add(Dropout(0.5))

In [97]:
def VGG16_arc():
  model=Sequential()
  ConvLayers(2,model,64)
  ConvLayers(2,model,128)
  ConvLayers(3,model,256)
  ConvLayers(3,model,512)
  ConvLayers(3,model,512)
  model.add(Flatten())
  FCLayers(model)
  FCLayers(model)
  model.add(Dense(37,activation='sigmoid'))
  #model.add(Dense(3,activation='softmax'))
  return model

In [98]:
from keras.models import load_model

In [99]:
#Model compilation
optimizer = keras.optimizers.Adam(lr=0.001, decay=5e-4)
model = VGG16_arc()
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy']) 
#model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [100]:
model.build(input_shape=(None,224,224,3))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_78 (ZeroPaddi (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 75, 75, 64)        1792      
_________________________________________________________________
zero_padding2d_79 (ZeroPaddi (None, 77, 77, 64)        0         
_________________________________________________________________
conv2d_79 (Conv2D)           (None, 25, 25, 64)        36928     
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 25, 25, 64)        0         
_________________________________________________________________
zero_padding2d_80 (ZeroPaddi (None, 27, 27, 64)        0         
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 9, 9, 128)        

In [36]:
import pandas as pd
df=pd.read_csv('/content/sample_data/Galaxy/processedtrainingdffinal.csv')
df = df.sample(frac=1).reset_index(drop=True)
df

,Unnamed: 0,GalaxyID,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,Class5.1,Class5.2,Class5.3,Class5.4,Class6.1,Class6.2,Class7.1,Class7.2,Class7.3,Class8.1,Class8.2,Class8.3,Class8.4,Class8.5,Class8.6,Class8.7,Class9.1,Class9.2,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
0,45145,758659resized2rotated_md_he.jpg,0.851461,0.097994,0.050544,0.000000,0.097994,0.000000,0.097994,0.000000,0.097994,0.048997,0.048997,0.000000,0.000000,0.425457,0.574543,0.477252,0.374209,0.000000,0.159546,0.000000,0.000000,0.000000,0.265911,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,58734,958094resized1rotated_md_he.jpg,0.833224,0.128460,0.038317,0.000000,0.128460,0.000000,0.128460,0.000000,0.128460,0.000000,0.063365,0.041725,0.023370,0.274457,0.725543,0.136510,0.696714,0.000000,0.000000,0.000000,0.000000,0.000000,0.082337,0.192120,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,27272,499446resized4rotated_md_he.jpg,0.041229,0.958771,0.000000,0.069167,0.889604,0.304257,0.585347,0.803795,0.085809,0.046031,0.428908,0.384381,0.030284,0.258017,0.741983,0.000000,0.027491,0.013738,0.062817,0.000000,0.058243,0.116486,0.000000,0.020471,0.000000,0.069167,0.0,0.000000,0.177535,0.244072,0.382188,0.025188,0.670281,0.035221,0.000000,0.0,0.073105
3,49474,820523resized2rotated_md_he.jpg,0.740947,0.243834,0.015219,0.000000,0.243834,0.000000,0.243834,0.000000,0.243834,0.000000,0.092333,0.054905,0.096597,0.000000,1.000000,0.585991,0.154956,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,45087,758103resized4rotated_md_he.jpg,0.659604,0.279778,0.060618,0.000000,0.279778,0.000000,0.279778,0.000000,0.279778,0.000000,0.000000,0.279778,0.000000,0.000000,1.000000,0.024229,0.618860,0.016516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246307,26374,485325resized1rotated_md_he.jpg,0.730657,0.246608,0.022735,0.000000,0.246608,0.000000,0.246608,0.000000,0.246608,0.000000,0.056785,0.189823,0.000000,0.083852,0.916148,0.460016,0.270641,0.000000,0.000000,0.000000,0.000000,0.027951,0.027951,0.027951,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
246308,55394,909361resized1rotated_md_he.jpg,0.042254,0.957746,0.000000,0.000000,0.957746,0.582791,0.374955,0.957746,0.000000,0.000000,0.506670,0.451076,0.000000,0.203781,0.796219,0.000000,0.042254,0.000000,0.058223,0.029112,0.058223,0.000000,0.029112,0.029112,0.000000,0.000000,0.0,0.000000,0.345032,0.575153,0.037561,0.035437,0.745126,0.106310,0.000000,0.0,0.070873
246309,46161,772901resized3rotated_md_he.jpg,0.360461,0.639539,0.000000,0.000000,0.639539,0.000000,0.639539,0.449606,0.189933,0.042870,0.596669,0.000000,0.000000,0.102590,0.897410,0.341357,0.019104,0.000000,0.000000,0.000000,0.068428,0.034162,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.166448,0.207291,0.075867,0.000000,0.000000,0.071883,0.076789,0.0,0.300934
246310,4123,161651resized1rotated_md_he.jpg,0.183494,0.775057,0.041449,0.000000,0.775057,0.048832,0.726225,0.364883,0.410174,0.198821,0.487204,0.000000,0.089031,0.471734,0.528266,0.020501,0.162993,0.000000,0.000000,0.000000,0.069750,0.354841,0.047143,0.000000,0.000000,0.000000,0.0,0.000000,0.173967,0.146341,0.044574,0.048791,0.146459,0.000000,0.076651,0.0,0.092981


In [37]:
classes = ['Class1.1', 'Class1.2', 'Class1.3', 'Class2.1', 'Class2.2', 'Class3.1','Class3.2', 'Class4.1', 'Class4.2', 'Class5.1', 'Class5.2', 'Class5.3','Class5.4', 'Class6.1', 'Class6.2', 'Class7.1', 'Class7.2', 'Class7.3','Class8.1', 'Class8.2', 'Class8.3', 'Class8.4', 'Class8.5', 'Class8.6', 'Class8.7', 'Class9.1', 'Class9.2', 'Class9.3', 'Class10.1', 'Class10.2', 'Class10.3', 'Class11.1', 'Class11.2', 'Class11.3', 'Class11.4', 'Class11.5', 'Class11.6'
]

In [38]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagenerator = ImageDataGenerator(validation_split=0.2)

train_generator = datagenerator.flow_from_dataframe(
    dataframe=df,
    directory="/content/gdrive/MyDrive/Processed/Data/he1/",
    x_col="GalaxyID",
    y_col=classes,
    subset="training",
    batch_size=64,
    shuffle=True,
    class_mode="raw",
    target_size=(224, 224))
valid_generator = datagenerator.flow_from_dataframe(
    dataframe=df,
    directory="/content/gdrive/MyDrive/Processed/Data/he1/",
    x_col="GalaxyID",
    y_col=classes,
    subset="validation",
    batch_size=64,
    shuffle=True,
    class_mode="raw",
    target_size=(224, 224))
train_step_size = train_generator.n // train_generator.batch_size
validation_step_size = valid_generator.n // valid_generator.batch_size

Found 197050 validated image filenames.
Found 49262 validated image filenames.


In [39]:
train_step_size

3078

In [40]:
validation_step_size

769

In [41]:
import h5py

In [42]:
import timeit
import matplotlib.pyplot as plt

In [101]:
model.load_weights('/content/gdrive/MyDrive/Processed_final.hdf5')

In [ ]:
from keras import callbacks
start = timeit.default_timer()
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="auto", patience = 3, verbose=2, 
                                        restore_best_weights = True)

checkpoint = callbacks.ModelCheckpoint(
    filepath='/content/gdrive/MyDrive/Processed/weights.hdf5', verbose=2, save_best_only=True)

new_classifier = model.fit(train_generator,
                           steps_per_epoch=train_step_size,
                           epochs=20,
                           validation_data=valid_generator,
                           validation_steps=validation_step_size,
                           callbacks =[checkpoint,earlystopping],
                           verbose=2,
                           )
end = timeit.default_timer()
print("Time Taken to run the model:",end - start, "seconds")

Epoch 1/20
3078/3078 - 958s - loss: 0.0133 - accuracy: 0.7272 - val_loss: 0.0121 - val_accuracy: 0.7434

Epoch 00001: val_loss improved from inf to 0.01214, saving model to /content/gdrive/MyDrive/Processed/weights.hdf5
Epoch 2/20
3078/3078 - 879s - loss: 0.0118 - accuracy: 0.7483 - val_loss: 0.0117 - val_accuracy: 0.7521

Epoch 00002: val_loss improved from 0.01214 to 0.01167, saving model to /content/gdrive/MyDrive/Processed/weights.hdf5
Epoch 3/20
3078/3078 - 873s - loss: 0.0112 - accuracy: 0.7599 - val_loss: 0.0112 - val_accuracy: 0.7568

Epoch 00003: val_loss improved from 0.01167 to 0.01125, saving model to /content/gdrive/MyDrive/Processed/weights.hdf5
Epoch 4/20
3078/3078 - 863s - loss: 0.0107 - accuracy: 0.7664 - val_loss: 0.0111 - val_accuracy: 0.7595

Epoch 00004: val_loss improved from 0.01125 to 0.01111, saving model to /content/gdrive/MyDrive/Processed/weights.hdf5
Epoch 5/20
3078/3078 - 861s - loss: 0.0104 - accuracy: 0.7717 - val_loss: 0.0110 - val_accuracy: 0.7619

Epo